In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [76]:
data = pd.read_excel('..\\Datos Partidos\\Gimnasia_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   42 non-null     datetime64[ns]
 1   Event                42 non-null     object        
 2   Session Name         42 non-null     object        
 3   Session Start        42 non-null     object        
 4   Session End          42 non-null     object        
 5   Tag Description      42 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            42 non-null     object        
 8   Tag End              42 non-null     object        
 9   Tag Duration (secs)  42 non-null     float64       
 10  Categoria            42 non-null     object        
 11  Zona                 42 non-null     object        
 12  Zona2                42 non-null     object        
 13  Accion               42 non-null     

In [77]:
data.head()

,Session Start Date,Event,Session Name,Session Start,Session End,Tag Description,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona,Zona2,Accion,Jugada
0,2022-06-07,Gimnasia,Situaciones,08:53:25,11:15:50,En Contra,NaN,00:34:09,00:34:11,2000.0,ATAQUE COMBINADO,4.0,3.0,Volver,1.0
1,2022-06-07,Gimnasia,Situaciones,08:53:25,11:15:50,En Contra,NaN,00:34:21,00:34:23,2000.0,ATAQUE COMBINADO,3.0,6.0,VOLVER,1.0
2,2022-06-07,Gimnasia,Situaciones,08:53:25,11:15:50,En Contra,NaN,00:34:29,00:34:31,2000.0,ATAQUE COMBINADO,6.0,2A,VOLVER,1.0
3,2022-06-07,Gimnasia,Situaciones,08:53:25,11:15:50,En Contra,NaN,00:34:33,00:34:35,2000.0,ATAQUE COMBINADO,2A,2B,GOL,1.0
4,2022-06-07,Gimnasia,Situaciones,08:53:25,11:15:50,A FAVOR,NaN,00:40:13,00:40:15,2000.0,PELOTA PARADA,8A,8B,REMATE BLOQUEADO,2.0


In [78]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [79]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [80]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque directo           3
          pelota parada            3
          transicion               1
en contra ataque combinado         1
          ataque directo           3
          pelota parada            3
          transicion               2

### Armo las coordenadas

In [81]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [82]:
data['Zona inicial'].value_counts()

3.0    6
6.0    6
2.1    6
8.1    5
5.0    5
4.0    4
7.0    4
9.0    2
1.0    2
8.2    1
2.2    1
Name: Zona inicial, dtype: int64

In [83]:
data['Zona Final'].value_counts()

2.2    9
8.2    7
2.1    6
3.0    4
8.1    4
6.0    3
5.0    3
9.0    2
7.0    2
4.0    1
1.0    1
Name: Zona Final, dtype: int64

In [84]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [85]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [86]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [87]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,3.0,136,45,135,27
1,6.0,45,136,27,27
2,2.1,136,70,27,81
3,2.2,70,30,81,81
4,8.2,255,207,81,81
5,8.1,207,255,81,81
6,5.0,255,136,81,81
7,2.1,136,70,81,81
8,2.2,70,30,81,81
9,5.0,136,136,135,81


In [88]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [89]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [90]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"136,135","45,27"
1,"45,27","136,27"
2,"136,27","70,81"
3,"70,81","30,81"
4,"255,81","207,81"
5,"207,81","255,81"
6,"255,81","136,81"
7,"136,81","70,81"
8,"70,81","30,81"
9,"136,135","136,81"


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                42 non-null     datetime64[ns]
 1   etiquetado           42 non-null     object        
 2   name                 42 non-null     object        
 3   Session Start        42 non-null     object        
 4   Session End          42 non-null     object        
 5   Evento               42 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            42 non-null     object        
 8   Tag End              42 non-null     object        
 9   Tag Duration (secs)  42 non-null     float64       
 10  Categoria            42 non-null     object        
 11  Zona inicial         42 non-null     object        
 12  Zona Final           42 non-null     object        
 13  Accion               42 non-null     

In [92]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Gimnasia'

In [73]:
data.to_excel("../Datos generados/reserva02-22-fecha1_gimnasia_situaciones.xlsx",index=False)